# My Tensorflow Note

### Key Concepts
- graph, tensor, operation, placeholder, variable

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
#reset the graph, preventing unknown error
tf.reset_default_graph()

# setting some constant
N, D, H = 64, 1000, 100

'''
define the very placeholder
 - using placeholder must pass feed_dict 
     while sess.run() cause "placeholder"
'''
x = tf.placeholder(tf.float32, shape=(N, D))
y = tf.placeholder(tf.float32, shape=(N, D))

'''
define Variable
 - A variable maintains state in the graph across calls to run()
 - variable need to be explicitly initialize !!
'''
w1 = tf.Variable(tf.random_normal((D, H)))
w2 = tf.Variable(tf.random_normal((H, D)))

#define forward & backward & weights update

#forward
h = tf.maximum(tf.matmul(x, w1), 0)
y_pred = tf.matmul(h, w2)

'''
#deprecated

diff = y_pred - y
loss = tf.reduce_mean(tf.reduce_sum(diff**2, axis=1))
'''
#use predifined losses
loss = tf.losses.mean_squared_error(y_pred, y)


'''
#deprecated

#assign new values to weights
#this is operations as well
#you can throw it into sess.run() as fetches

grad_w1, grad_w2 = tf.gradients(loss, [w1, w2])
learning_rate = 1e-5
new_w1 = w1.assign(w1 - learning_rate * grad_w1)
new_w2 = w2.assign(w2 - learning_rate * grad_w2)
updates = tf.group(new_w1, new_w2)
'''

'''
using predifined optimizer to minimize loss 
 - minimize() here is a combination of compute_gradients() & apply_gradients()
 - compute_gradients(loss) can compute the gradients of loss 
    for the variables in var_list(default is the global variables )
 - apply_gradient() just as its name, return the operation of applying gradients 
'''
optimizer = tf.train.GradientDescentOptimizer(1e-3)
updates = optimizer.minimize(loss)

with tf.Session() as sess:
    '''
    run() 
     - it can runs operations and evaluates tensors offered in fetches
     - fetches takes in graph elememts such as tf.Tensor, tf.Operation
         so fetches is loss , updates here
     - this method only run one time, hence the loop
     - return is the values you require to compute in the fetches, 
     - return has the same shape as fetches, but int form of list or int
    '''
    
    '''
    feed_dict
     - subtitute the placeholder define above
     - key corresponds to placeholder above !!!
     - value corresponds to values you want to assign as np array
    '''
    
    
    values = {
        x:np.random.randn(N, D),
        y:np.random.randn(N, D),
    }
    #initialize global variable, passing the operation here
    sess.run(tf.global_variables_initializer())
    losses = []
    for t in range(50):
        # _ here is tmp var which will not be used in the future
        loss_val, _ = sess.run(fetches=[loss, updates], feed_dict=values)
        losses.append(loss_val)

In [7]:
#reset the graph, preventing unknown error
tf.reset_default_graph()

N, D, H = 64, 1000, 100

#using gpu when performing below operations!
#actually tensorflow can do assignment on gpu automatically
#so this line is just as an reminder
with tf.device('/gpu:0'):
    x = tf.placeholder(tf.float32, shape=(N, D))
    y = tf.placeholder(tf.float32, shape=(N, D))

    #use a xavier init
    init = tf.contrib.layers.xavier_initializer()
    #use predefined layers, this automatically sets up weight & bias
    h = tf.layers.dense(inputs=x, units=H, activation=tf.nn.relu,
                       kernel_initializer=init)
    tf.summary.histogram('h', h)
    y_pred = tf.layers.dense(inputs=h, units=D, 
                             kernel_initializer=init)

    loss = tf.losses.mean_squared_error(y_pred, y)
    tf.summary.scalar('loss', loss)
    
    merge_summary = tf.summary.merge_all()
    
    optimizer = tf.train.GradientDescentOptimizer(1e-0)
    updates = optimizer.minimize(loss)


saver = tf.train.Saver()
#config allow_soft_placement here is to let the computation fall back to cpu version
#in this case, the tf.losses.mean_squared_error has no corresponding gpu version
config=tf.ConfigProto(allow_soft_placement=True)
config.allow_soft_placement = True
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    values = {
        x:np.random.randn(N, D),
        y:np.random.randn(N, D),
    }
    sess.run(tf.global_variables_initializer())
    #absolute path here!!!
    summary_writer = tf.summary.FileWriter(logdir='/home/jacktroy/JupyterNotebook/Tutorial/eventfiles/',
                                         graph=sess.graph)
    for t in range(1000):
        loss_val, _ = sess.run(fetches=[loss, updates], feed_dict=values)
        #summary_str = sess.run(merge_summary, feed_dict=values)
        #summary_writer.add_summary(summary=summary_str, global_step=t)
    
    #save_model
    model_path = '/home/jacktroy/JupyterNotebook/Tutorial/my_model/test'
    save_path = saver.save(sess, save_path=model_path)
    

with tf.Session(config=config) as sess:
    print('load up the model')
    #sess.run(tf.global_variables_initializer())
    saver.restore(sess, save_path=model_path)
    
    for t in range(50):
        loss_val, _ = sess.run(fetches=[loss, updates], feed_dict=values)
        


load up the model
INFO:tensorflow:Restoring parameters from /home/jacktroy/JupyterNotebook/Tutorial/my_model/test


In [ ]:
tf.reset_default_graph()
with tf.device('/gpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
  c = tf.matmul(a, b)

# 通过log_device_placement指定在日志中输出变量和操作所在的设备
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

#seems like log only appear using terminal, sad
print(sess.run(c))

In [9]:
#useful function

#datatype conversion, tf.cast(x, dtype, name=None)
tf.cast(tf.constant([1.0]), tf.int32)

#reshape, tf.reshape(a, shape = [])
# tensor 't' is [1, 2, 3, 4, 5, 6, 7, 8, 9]
# tensor 't' has shape [9]
tf.reshape(tf.constant(np.arange(1,10)), [3, 3])
#note that using '-1' to infer, you need to follow its order to reshape
#reshape (H, W, C) to (H * W, C), the shape param is this [-1, C]
#can't reshape (H, W, C) to (C, H * W) using shape=[C, -1]
#should use tf.transpose instead

<tf.Tensor 'Reshape_2:0' shape=(3, 3) dtype=int64>